# Modelagem Preditiva (N2)

---

## 🗂️ [ Projeto ]

**Dataset:** [Nome do dataset ou fonte de dados]  
**Objetivo Geral:** [Breve explicação do propósito — ex.: “Aplicar técnicas de modelagem preditiva para identificar padrões e realizar previsões a partir dos dados tratados.”]  

---

## 🎯 Propósito do Notebook

Este notebook representa a **segunda etapa do pipeline analítico**, voltada à **modelagem preditiva** e **validação estatística**.  
A partir dos dados já preparados e consolidados no **N1 (Preparação de Dados)**, são aplicadas técnicas de **Machine Learning supervisionado** ou **não supervisionado**, de acordo com o tipo de problema em estudo (classificação, regressão ou agrupamento).

---

## 📋 Descrição Geral das Etapas

Nesta fase, o foco é **construir, testar e comparar modelos preditivos**, assegurando desempenho consistente e interpretabilidade.  
A execução segue as boas práticas de **ciência de dados reprodutível** e utiliza a **estrutura modular do Template de Dados**, garantindo clareza e padronização entre projetos.

**Tarefas previstas:**  
- Importação dos artefatos e metadados gerados no N1.  
- Separação entre **variáveis independentes (features)** e **variável-alvo (target)**, quando aplicável.  
- Divisão do dataset em **conjuntos de treino e teste**.  
- Construção de **pipelines de modelagem** com etapas de pré-processamento, ajuste e avaliação.  
- Treinamento de múltiplos algoritmos (ex.: *Regressão Logística, Random Forest, KNN, Árvores de Decisão, Regressão Linear, Clustering, etc.*).  
- Avaliação comparativa de desempenho com métricas adequadas (Acurácia, F1-Score, RMSE, Silhouette Score, entre outras).  
- Interpretação dos resultados e análise de importância das variáveis.  
- Exportação de modelos, métricas e relatórios para o **N3 (Análise e Insights)**.

---

> 💡 **Resumo:**  
> O N2 tem como objetivo transformar o dataset preparado no N1 em uma base analítica pronta para modelagem preditiva, explorando algoritmos, métricas e estratégias de validação.  
> É o núcleo experimental do projeto — onde hipóteses são testadas, modelos são ajustados e o conhecimento começa a emergir de forma quantitativa, sustentando a próxima etapa: o **N3 (Geração de Insights e Visualizações)**.


# 📘 Bootstrap do N2 — Inicialização do Ambiente e Configuração do Dataset

Esta etapa realiza o **bootstrap completo do N2**, responsável por preparar o ambiente,
carregar as configurações globais e disponibilizar o dataset processado para o início da modelagem supervisionada.

---

## ⚙️ Etapas executadas

1. **Localização automática da raiz do projeto (`PROJECT_ROOT`)**  
   O código percorre a árvore de diretórios até encontrar o arquivo `config/defaults.json`.  
   Isso garante que o notebook funcione corretamente mesmo quando aberto a partir de subpastas.

2. **Injeção da raiz e do módulo `utils/` no `sys.path`**  
   Essa etapa assegura que funções utilitárias, como as do arquivo `utils/utils_data.py`,
   possam ser importadas e utilizadas sem ajustes manuais de caminho.

3. **Carregamento das configurações globais (`defaults.json` + `local.json`)**  
   São lidas definições essenciais — como nome da variável alvo (`target_column`), 
   proporção de treino e teste (`test_size`), controle de aleatoriedade (`random_state`),
   estratégias de codificação, normalização e detecção de outliers.

4. **Validação da estrutura de diretórios padrão**  
   Pastas como `artifacts/`, `reports/` e `artifacts/models/` são verificadas e criadas automaticamente,  
   garantindo que o pipeline tenha onde salvar seus resultados e metadados.

5. **Descoberta e leitura do dataset processado**  
   O notebook identifica o arquivo final exportado pelo N1 (geralmente `data/processed/processed.parquet`),  
   realiza a leitura conforme o formato (Parquet, CSV ou Excel) e carrega o conteúdo em um `DataFrame` pandas.

6. **Identificação da variável alvo e separação de dados**  
   A coluna-alvo é localizada automaticamente com base nas configurações, e o conjunto é dividido em:  
   - `X`: features preditoras  
   - `y`: variável alvo (`TARGET_COL`)

7. **Verificações de sanidade e distribuição da target**  
   São exibidas métricas básicas como quantidade de linhas, colunas, valores nulos e distribuição da variável alvo.  
   Essas informações permitem validar rapidamente se o dataset está íntegro e pronto para o treinamento.

---

> 💡 **Em resumo:**  
> Esta célula garante que o N2 sempre inicie em um ambiente estável e configurado,  
> com o dataset processado e a variável-alvo corretamente carregados,  
> prontos para o pipeline de modelagem supervisionada.


In [4]:
# -*- coding: utf-8 -*-
from __future__ import annotations

from pathlib import Path
import sys
import importlib

def _find_up(relative_path: str, start: Path | None = None) -> Path | None:
    start = start or Path.cwd()
    rel = Path(relative_path)
    for base in (start, *start.parents):
        cand = base / rel
        if cand.exists():
            return cand
    return None

_cfg = _find_up("config/defaults.json")
if _cfg is None:
    raise FileNotFoundError(
        "config/defaults.json não encontrado. Abra o notebook N2 dentro da estrutura do projeto."
    )

PROJECT_ROOT = _cfg.parent.parent.resolve()
UTILS_DIR = PROJECT_ROOT / "utils"
INIT_FILE = UTILS_DIR / "__init__.py"

if not UTILS_DIR.exists():
    raise ModuleNotFoundError(f"Pasta não encontrada: {UTILS_DIR}")
if not INIT_FILE.exists():
    INIT_FILE.write_text("", encoding="utf-8")

root_str = str(PROJECT_ROOT)
if root_str not in sys.path:
    sys.path.insert(0, root_str)

print(f"[INFO] PROJECT_ROOT: {PROJECT_ROOT}")
print(f"[INFO] sys.path ok. utils: {UTILS_DIR}")

import utils.utils_data as ud
importlib.reload(ud)  # garante versão mais recente se você editar utils_data.py


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] sys.path ok. utils: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\utils


<module 'utils.utils_data' from 'C:\\Users\\fabio\\Projetos DEV\\data projects\\telco-customer-churn-analysis\\utils\\utils_data.py'>

In [5]:
# 1) Puxa o contexto completo
ctx = ud.n2_bootstrap_context()

# 2) Descompacta o que vai usar nas próximas células
PROJECT_ROOT   = ctx["project_root"]
cfg            = ctx["cfg"]
df             = ctx["df"]
X              = ctx["X"]
y              = ctx["y"]
TARGET_COL     = ctx["target_col"]
processed_path = ctx["processed_path"]
num_cols       = ctx["num_cols"]
cat_cols       = ctx["cat_cols"]
other_cols     = ctx["other_cols"]
test_size      = ctx["test_size"]
random_state   = ctx["random_state"]
scale_numeric  = ctx["scale_numeric"]

print(f"[INFO] Processed file: {processed_path}")
print(f"[INFO] Target: {TARGET_COL}")
print(f"[INFO] Shapes -> X: {X.shape} | y: {y.shape}")

# 3) Painel estilizado N2 
ud.n2_render_status_panel(ctx, keep_path_parts=4)


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] [ensure_dirs] artifacts=C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\artifacts | reports=C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports | models=C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\artifacts\models
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] Processed file: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\data\processed\processed.parquet
[INFO] Target: Churn
[INFO] Shapes -> X: (7043, 132) | y: (7043,)


In [6]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,PaymentMethod_len,PaymentMethod_word_count,PaymentMethod_has_error,PaymentMethod_has_cancel,PaymentMethod_has_premium,Churn_len,Churn_word_count,Churn_has_error,Churn_has_cancel,Churn_has_premium
0,7590-VHVEG,female,0,yes,no,1,no,no,dsl,no,...,16,2,False,False,False,2,1,False,False,False
1,5575-GNVDE,male,0,no,no,34,yes,no,dsl,yes,...,12,2,False,False,False,2,1,False,False,False
2,3668-QPYBK,male,0,no,no,2,yes,no,dsl,yes,...,12,2,False,False,False,3,1,False,False,False
3,7795-CFOCW,male,0,no,no,45,no,no,dsl,yes,...,25,3,False,False,False,2,1,False,False,False
4,9237-HQITU,female,0,no,no,2,yes,no,fiber optic,no,...,16,2,False,False,False,3,1,False,False,False


# ✂️ Split Treino/Teste e Resumo de Colunas

Nesta etapa, preparamos os dados para a modelagem supervisionada, separando variáveis independentes (`X`) da variável alvo (`y`) e realizando o **split treino/teste** com parâmetros consistentes definidos no `defaults.json`.

## O que acontece aqui

1. **Separação de X e y**
   - Remove-se a coluna alvo (`TARGET_COL`) de `X` e mantém-se `y` intacta para preservar a distribuição original.

2. **Resumo de tipos de colunas**
   - Gera contagens de colunas **numéricas**, **categóricas** e **outras** (se houver).
   - Este diagnóstico orienta a construção do `ColumnTransformer` na próxima etapa.

3. **Split treino/teste com estratificação**
   - Usa `test_size` e `random_state` da configuração.
   - Ativa `stratify=y` sempre que houver mais de uma classe, mantendo a **mesma proporção** de classes em `train` e `test`.

4. **Diagnósticos úteis**
   - Compara a **distribuição da target** no conjunto geral vs. `train` vs. `test` (contagem e %).
   - Sinaliza **desbalanceamento** quando a classe majoritária ultrapassa um limite (ex.: 80%).
   - Detecta **categorias raras** (freq. < 5 no `train`) em variáveis categóricas — útil para evitar explosão do One-Hot e para tratar identificadores.

## Observações frequentes

- **IDs como `customerID`** aparecem como inúmeras categorias raras. Em geral, **não devem ser usadas como preditores**, pois são identificadores sem relação causal. Recomenda-se **removê-las** de `X` (ou excluí-las da lista de categóricas antes do One-Hot).
- Se houver **alto cardinalidade** e a variável for preditiva, considere técnicas específicas (target encoding, hashing, catboost encoder). Neste template, mantemos a abordagem pedagógica e transparente com One-Hot e remoção de IDs.

## Próximo passo

- Construir o **pré-processamento** com `ColumnTransformer` (imputação + One-Hot para categóricas + escala opcional para numéricas) e ajustar no conjunto de treino.


In [8]:
# === Split treino/teste e resumo de colunas (diagnóstico reforçado) ===
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# Resumo de colunas do X
num_cols, cat_cols, other_cols = summarize_columns(X)
print(f"[INFO] Colunas numéricas: {len(num_cols)} | categóricas: {len(cat_cols)} | ignoradas: {len(other_cols)}")

# Parâmetros do split (eco explícito)
test_size = float(cfg.get("test_size", 0.2))
random_state = int(cfg.get("random_state", 42))
do_stratify = (pd.Series(y).nunique() > 1)
print(f"[INFO] Split params -> test_size={test_size} | random_state={random_state} | stratify={do_stratify}")

# Split com estratificação quando aplicável
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=test_size,
    random_state=random_state,
    stratify=y if do_stratify else None,
)
print(f"[INFO] X_train: {X_train.shape} | X_test: {X_test.shape}")

# Conferência da proporção da target no geral vs train/test
def _dist(s):
    vc = s.value_counts(dropna=False)
    pct = (vc / len(s) * 100).round(2)
    return pd.DataFrame({"count": vc, "pct": pct})

print("\n[CHECK] Distribuição da target — geral / train / test")
disp_overall = _dist(y)
disp_train   = _dist(y_train)
disp_test    = _dist(y_test)
print("• Geral:\n", disp_overall.to_string())
print("• Train:\n", disp_train.to_string())
print("• Test:\n", disp_test.to_string())

# Alerta simples de desbalanceamento (macro-level)
imbalance_threshold = 0.80  # ajuste se quiser
top_ratio = disp_overall["pct"].max() / 100.0
if top_ratio >= imbalance_threshold:
    print(f"[AVISO] Target potencialmente desbalanceada (classe majoritária ~{top_ratio:.1%}). "
          "Considere métricas robustas (F1, ROC-AUC), validação estratificada e/ou técnicas de balanceamento.")

# Sinalização de categorias raras (pode explodir OHE)
rare_threshold = 5  # frequência mínima
rare_report = {}
for c in cat_cols:
    cnt = X_train[c].value_counts(dropna=False)
    rare = cnt[cnt < rare_threshold]
    if len(rare) > 0:
        rare_report[c] = len(rare)
if rare_report:
    top_rare_cols = sorted(rare_report.items(), key=lambda kv: kv[1], reverse=True)[:10]
    print(f"[INFO] Colunas categóricas com categorias raras (<{rare_threshold} amostras) no train:")
    for col, n_rare in top_rare_cols:
        print(f" - {col}: {n_rare} categorias raras")
else:
    print("[INFO] Nenhuma categoria rara detectada no train com o limite atual.")


[INFO] Colunas numéricas: 62 | categóricas: 65 | ignoradas: 0
[INFO] Split params -> test_size=0.2 | random_state=42 | stratify=True
[INFO] X_train: (5634, 127) | X_test: (1409, 127)

[CHECK] Distribuição da target — geral / train / test
• Geral:
        count    pct
Churn              
no      5174  73.46
yes     1869  26.54
• Train:
        count    pct
Churn              
no      4139  73.46
yes     1495  26.54
• Test:
        count    pct
Churn              
no      1035  73.46
yes      374  26.54
[INFO] Colunas categóricas com categorias raras (<5 amostras) no train:
 - customerID: 5634 categorias raras


In [9]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,PaymentMethod_len,PaymentMethod_word_count,PaymentMethod_has_error,PaymentMethod_has_cancel,PaymentMethod_has_premium,Churn_len,Churn_word_count,Churn_has_error,Churn_has_cancel,Churn_has_premium
0,7590-VHVEG,female,0,yes,no,1,no,no phone service,dsl,no,...,16,2,False,False,False,2,1,False,False,False
1,5575-GNVDE,male,0,no,no,34,yes,no,dsl,yes,...,12,2,False,False,False,2,1,False,False,False
2,3668-QPYBK,male,0,no,no,2,yes,no,dsl,yes,...,12,2,False,False,False,3,1,False,False,False
3,7795-CFOCW,male,0,no,no,45,no,no phone service,dsl,yes,...,25,3,False,False,False,2,1,False,False,False
4,9237-HQITU,female,0,no,no,2,yes,no,fiber optic,no,...,16,2,False,False,False,3,1,False,False,False


# ⚙️ Pré-processamento — One-Hot denso + escala opcional

Nesta etapa, preparamos `X` para treinamento aplicando **imputação**, **codificação categórica** e **padronização opcional**.
Mantemos o bloco **visível no N2** por transparência pedagógica: quem lê o notebook consegue enxergar claramente *como* os dados chegam ao modelo.

## O que acontece aqui

1. **Detecção dos conjuntos de colunas**
   - Usamos `num_cols` e `cat_cols` (derivados de `summarize_columns(X)`) para endereçar o tratamento adequado a cada tipo.

2. **Pipeline numérico**
   - `SimpleImputer(strategy="mean")` para preencher valores ausentes.
   - `StandardScaler()` **opcional** (controlado por `cfg["scale_numeric"]`), deixando as features numéricas com média 0 e desvio 1.
   - Este bloco é montado dinamicamente: se `scale_numeric=False`, o scaler é omitido.

3. **Pipeline categórico**
   - `SimpleImputer(strategy="most_frequent")` para preencher categorias ausentes.
   - `OneHotEncoder(handle_unknown="ignore", output denso)`. O código é **compatível** com versões antigas/novas do scikit-learn:
     - usa `sparse_output=False` quando disponível;
     - faz *fallback* para `sparse=False` em versões anteriores.

4. **ColumnTransformer**
   - Une os dois pipelines e aplica a transformação **apenas às colunas selecionadas**, descartando o restante (`remainder="drop"`).
   - Mantemos `verbose_feature_names_out=False` para preservar nomes legíveis nas colunas expandidas do One-Hot.

5. **Ajuste e transformação**
   - `fit` no `X_train` (usando `y_train` quando necessário) e `transform` no `X_train`/`X_test`.
   - Garantimos saída **densa** (arrays NumPy), apropriada para modelos que não aceitam matrizes esparsas.

6. **Diagnósticos rápidos**
   - Imprimimos:
     - `scale_numeric`, tamanho de `num_cols`/`cat_cols`,
     - quantidade total de **features transformadas**,
     - *preview* dos **primeiros nomes** de features,
     - `shapes` de `X_train_t`/`X_test_t`,
     - **memória estimada** (MB) dos arrays transformados.

## Observações importantes

- **Identificadores (IDs)** como `customerID` não devem compor `cat_cols` (explodem o One-Hot e não agregam sinal preditivo).  
  Remova-os de `X` *antes* do preprocess ou exclua-os da lista de categóricas.
- Para colunas categóricas com **alta cardinalidade**, considere alternativas como **hashing trick**, **target encoding** ou **CatBoost encoding** em versões futuras do template.
- Se a memória ficar alta, avalie:
  - reduzir cardinalidade (agrupar categorias raras),
  - trocar o encoder,
  - ou trabalhar com saída **esparsa** (e modelos que aceitem esparsidade).

## Próximo passo

- Encaixar o `preprocess` nos **Pipelines** dos modelos (Dummy, LogisticRegression, KNN, RandomForest, etc.) e seguir para o **seletor de modelos e hiperparâmetros (UI)**.


In [10]:
# === Pré-processamento (One-Hot denso + escala opcional) ===
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd

# Função compatível com versões antigas/novas do scikit-learn
def build_preprocess(numeric_cols, categorical_cols, scale_numeric=True):
    """
    Cria um ColumnTransformer com:
      - Numéricas: imputação média + (opcional) StandardScaler
      - Categóricas: imputação mais frequente + OneHotEncoder denso (compatível com versões)
    """
    # 1) OneHotEncoder compatível com sklearn >= 1.4 (sparse_output) e versões anteriores (sparse)
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

    # 2) Pipeline numérico
    num_steps = [("imputer", SimpleImputer(strategy="mean"))]
    if scale_numeric and len(numeric_cols) > 0:
        num_steps.append(("scaler", StandardScaler()))
    numeric_transformer = Pipeline(steps=num_steps)

    # 3) Pipeline categórico
    cat_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", ohe),
    ])

    # 4) ColumnTransformer
    ct = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_cols if len(numeric_cols) > 0 else []),
            ("cat", cat_transformer, categorical_cols if len(categorical_cols) > 0 else []),
        ],
        remainder="drop",
        verbose_feature_names_out=False,
    )
    return ct


# ---- Construção do pré-processador conforme config
scale_numeric = bool(cfg.get("scale_numeric", True))
preprocess = build_preprocess(num_cols, cat_cols, scale_numeric=scale_numeric)
print(f"[INFO] scale_numeric={scale_numeric} | num_cols={len(num_cols)} | cat_cols={len(cat_cols)}")

# ---- Ajuste no treino
preprocess.fit(X_train, y_train)

# ---- Tentativa de inspecionar nomes de features geradas
feat_names = None
try:
    feat_names = preprocess.get_feature_names_out()
    print(f"[INFO] Features transformadas: {len(feat_names)}")
    print("[INFO] Preview das primeiras 20 features:")
    for n in feat_names[:20]:
        print(" -", n)
except Exception as e:
    print(f"[WARN] Não foi possível obter nomes de features: {e}")

# ---- Transformação de treino e teste (densa)
X_train_t = preprocess.transform(X_train)
X_test_t  = preprocess.transform(X_test)

# Garante array denso (caso algum backend retorne esparso)
if hasattr(X_train_t, "toarray"):
    X_train_t = X_train_t.toarray()
if hasattr(X_test_t, "toarray"):
    X_test_t = X_test_t.toarray()

print(f"[INFO] Shapes transformados -> X_train_t: {X_train_t.shape} | X_test_t: {X_test_t.shape}")

# ---- (Opcional) Relato rápido de memória
def _mb(nbytes): 
    return f"{(nbytes or 0) / (1024**2):.2f} MB"

try:
    mem_train = X_train_t.nbytes if isinstance(X_train_t, np.ndarray) else None
    mem_test  = X_test_t.nbytes  if isinstance(X_test_t,  np.ndarray) else None
    print(f"[INFO] Memória estimada -> train={_mb(mem_train)} | test={_mb(mem_test)}")
except Exception:
    pass

# ---- (Opcional) DataFrame de features apenas para DEBUG/inspeção (cuidado com memória!)
# Ative só quando necessário; por padrão, mantemos como arrays numpy eficientes.
# if feat_names is not None:
#     X_train_df = pd.DataFrame(X_train_t, columns=feat_names, index=X_train.index)
#     X_test_df  = pd.DataFrame(X_test_t,  columns=feat_names, index=X_test.index)
#     display(X_train_df.head(3))


[INFO] scale_numeric=True | num_cols=62 | cat_cols=65
[INFO] Features transformadas: 5787
[INFO] Preview das primeiras 20 features:
 - SeniorCitizen
 - tenure
 - MonthlyCharges
 - TotalCharges
 - SeniorCitizen_was_missing
 - tenure_was_missing
 - MonthlyCharges_was_missing
 - TotalCharges_was_missing
 - customerID_was_missing
 - gender_was_missing
 - Partner_was_missing
 - Dependents_was_missing
 - PhoneService_was_missing
 - MultipleLines_was_missing
 - InternetService_was_missing
 - OnlineSecurity_was_missing
 - OnlineBackup_was_missing
 - DeviceProtection_was_missing
 - TechSupport_was_missing
 - StreamingTV_was_missing
[INFO] Shapes transformados -> X_train_t: (5634, 5787) | X_test_t: (1409, 5787)
[INFO] Memória estimada -> train=248.75 MB | test=62.21 MB


## 4) Seletor de modelos e hiperparâmetros (UI)

In [11]:
# === UI: Seletor de modelos + Hyperdrive (encapsulado no utils_data) ===
from utils.utils_data import n2_build_models_ui

n2_build_models_ui(
    preprocess=preprocess,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    models_dir=models_dir,
    reports_dir=reports_dir
)


Box(children=(VBox(children=(HBox(children=(HTML(value="<div class='lumen-title'>Seletor de modelos · Hyperdri…

## 5) Treino, avaliação e export de artefatos

In [ ]:

def collect_params_from_tab():
    return {name: {k: w.value for k, w in spec["params"].items()} for name, spec in MODEL_REGISTRY.items()}

def compute_and_plot(pipe, name, X_test, y_test):
    y_pred = pipe.predict(X_test)
    metrics = compute_metrics(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(5,4))
    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred)).plot(ax=ax, colorbar=False)
    ax.set_title(f"Matriz de confusão — {name}")
    plt.show()
    try_plot_roc(pipe, X_test, y_test)
    print(f"[OK] {name}: accuracy={metrics['accuracy']:.4f} | f1={metrics['f1']:.4f}")
    return metrics

def train_and_eval(models_selected, params_by_model):
    results = {}
    for name, selected in models_selected.items():
        if not selected: 
            continue
        ModelClass = MODEL_REGISTRY[name]["class"]
        params = params_by_model.get(name, {})
        pipe = Pipeline(steps=[("preprocess", preprocess), ("clf", ModelClass(**params))])
        pipe.fit(X_train, y_train)
        metrics = compute_and_plot(pipe, name, X_test, y_test)
        results[name] = {"pipeline": pipe, "metrics": metrics, "params": params}
    return results

@out.capture()
def on_train_clicked(_):
    clear_output(wait=True)
    display(W.HTML("<h4>Treinando...</h4>"))
    selected = {name: chk.value for name, chk in model_checks.items()}
    params = collect_params_from_tab()
    results = train_and_eval(selected, params)

    if results:
        df_rank = pd.DataFrame([{"model": k, **v["metrics"], **{"params": v["params"]}} for k, v in results.items()])\
                    .sort_values(by=["f1", "accuracy"], ascending=False)
        display(W.HTML("<h4>Ranking (F1, depois Accuracy)</h4>")); display(df_rank)
        if cb_persist.value:
            for name, rec in results.items():
                persist_artifacts(name, rec["pipeline"], rec["metrics"], rec["params"], models_dir, reports_dir)
    else:
        print("[AVISO] Nenhum modelo selecionado.")

btn_train.on_click(on_train_clicked)
